In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Boltzman Machine

## Pre-Processing

In [11]:
colnames = ['user_id', 'movie_id', 'rating', 'time']
training_set = pd.read_csv('./data/ml-100k/u1.base', delimiter = '\t', header=None, names=colnames)
test_set = pd.read_csv('./data/ml-100k/u1.test', delimiter = '\t', header=None, names=colnames)
full_set = pd.concat([training_set, test_set])
full_set['time'] = pd.to_datetime(full_set['time'], unit='s')
full_set.head()

,user_id,movie_id,rating,time
0,1,1,5,1997-09-22 22:02:38
1,1,2,3,1997-10-15 05:26:11
2,1,3,4,1997-11-03 07:42:40
3,1,4,3,1997-10-15 05:25:19
4,1,5,3,1998-03-13 01:15:12


In [12]:
nb_users = full_set['user_id'].nunique()
nb_movies = full_set['movie_id'].nunique()

In [13]:
training_set['rating'] = training_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
test_set['rating'] = test_set['rating'].apply(lambda x: -1 if x < 1 else(0 if x < 3 else 1))
training_set = training_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.groupby(['user_id', 'movie_id'])['rating'].max().unstack().fillna(-1)
test_set = test_set.reindex(columns = training_set.columns.tolist()).fillna(-1)
training_set.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.0,1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [14]:
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Model Training

In [6]:
# boltman machine class
class RBM():
    '''
    Parameters:
    nv: int. Number of Visible Nodes, supposed to be the size of data
    nh: int. Number of Hidden Nodes
    '''
    def __init__(self, nv, nh):
        # Initialize 2D weight, bias of Hidden Node by a random number
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        # Initialize weight, bias of Visible Node by a random number
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        '''
        To get probability of hidden node given visible node
        Parameters:
        x: int. Vector of neurons
        '''
        # weight multiply neuron
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        # To get probability of visible node given hidden node
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        # contrast divergence algorithm
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

batch_size = 100
rbm = RBM(len(training_set[0]), 100)

In [7]:
# model training
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.0
    
    # Gibb Sampling
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        # monte carlo
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3438)
epoch: 2 loss: tensor(0.2502)
epoch: 3 loss: tensor(0.2472)
epoch: 4 loss: tensor(0.2476)
epoch: 5 loss: tensor(0.2508)
epoch: 6 loss: tensor(0.2481)
epoch: 7 loss: tensor(0.2483)
epoch: 8 loss: tensor(0.2471)
epoch: 9 loss: tensor(0.2464)
epoch: 10 loss: tensor(0.2443)


## Evaluation

In [8]:
# testing
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2484)


# AutoEncoder

## Pre-processing

In [18]:
training_set = pd.read_csv('./data/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('./data/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [19]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1] [data[:, 0] == id_users]
        id_ratings = data[:, 2] [data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [20]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Model Training

In [21]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [22]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7711)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0308)
epoch: 6loss: tensor(1.0265)
epoch: 7loss: tensor(1.0237)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0189)
epoch: 12loss: tensor(1.0182)
epoch: 13loss: tensor(1.0177)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0171)
epoch: 16loss: tensor(1.0171)
epoch: 17loss: tensor(1.0169)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0160)
epoch: 21loss: tensor(1.0158)
epoch: 22loss: tensor(1.0158)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0154)
epoch: 25loss: tensor(1.0157)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0152)
epoch: 28loss: tensor(1.0152)
epoch: 29loss: tensor(1.0139)
epoch: 30loss: tensor(1.0120)
epoch: 31loss: tensor(1.0105)
epoch: 32loss: tensor(1.0082)
epoch: 33loss: tensor(1.0058)
epoch: 34loss: tens

## Evaluation

In [23]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9548)
